In [ ]:
!unzip -qq '/content/drive/My Drive/color.zip'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_transforms = transforms.Compose([
                            transforms.Resize(224),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(224, padding = 10),
                            transforms.ToTensor()
                                      ])
train_data = datasets.ImageFolder(root = '/content/color',transform=train_transforms)
RATIO = 0.8
n_train_examples = int(len(train_data) * RATIO)
n_orginalvalid_examples = len(train_data) - n_train_examples
n_valid_examples = int((n_orginalvalid_examples)*0.5)
n_test_examples = n_orginalvalid_examples - n_valid_examples
print(n_train_examples)
print(n_valid_examples)
print(n_test_examples)

train_data, valid_data, test_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples, n_test_examples])

43444
5430
5431


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import copy
import random
import time
import torchvision.models as models


In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features,out_features=38)

for param in model.parameters():
  param.requires_grad=True
    


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [ ]:
model = model.to(device)

In [ ]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,272,550 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),weight_decay=0.01, lr=0.0001)
criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 200

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 47s
	Train Loss: 0.361 | Train Acc: 93.19%
	 Val. Loss: 0.147 |  Val. Acc: 97.44%
Epoch: 02 | Epoch Time: 4m 47s
	Train Loss: 0.331 | Train Acc: 94.23%
	 Val. Loss: 0.152 |  Val. Acc: 97.17%
Epoch: 03 | Epoch Time: 4m 48s
	Train Loss: 0.318 | Train Acc: 94.85%
	 Val. Loss: 0.145 |  Val. Acc: 97.61%
Epoch: 04 | Epoch Time: 4m 48s
	Train Loss: 0.305 | Train Acc: 95.11%
	 Val. Loss: 0.136 |  Val. Acc: 97.85%
Epoch: 05 | Epoch Time: 4m 48s
	Train Loss: 0.304 | Train Acc: 95.10%
	 Val. Loss: 0.132 |  Val. Acc: 97.90%
Epoch: 06 | Epoch Time: 4m 47s
	Train Loss: 0.292 | Train Acc: 95.52%
	 Val. Loss: 0.130 |  Val. Acc: 97.96%
Epoch: 07 | Epoch Time: 4m 49s
	Train Loss: 0.290 | Train Acc: 95.49%
	 Val. Loss: 0.123 |  Val. Acc: 97.87%
Epoch: 08 | Epoch Time: 4m 53s
	Train Loss: 0.287 | Train Acc: 95.58%
	 Val. Loss: 0.118 |  Val. Acc: 98.22%
Epoch: 09 | Epoch Time: 4m 47s
	Train Loss: 0.283 | Train Acc: 95.71%
	 Val. Loss: 0.119 |  Val. Acc: 98.03%
Epoch: 10 | Epoch T

KeyboardInterrupt: ignored

In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, '/content/drive/My Drive/model123.pt')